# Classificador de notícies

En aquesta pràctica, crearem un classificador de notícies utilitzant les tècniques de processament de llenguatge natural que hem vist a classe, centrades en la representació del text.

Utilitzarem el `dataset` [AG News](https://www.kaggle.com/amananandrai/ag-news-classification-dataset) que conté 1.000.000 de notícies de 4 categories diferents.

## Dataset

Per carregar el dataset, utilitzarem la llibreria `datasets`. Aquesta llibreria ens permetrà carregar molts datasets diferents de manera senzilla. En aquest cas, carregarem el dataset d'AG News.

## Preparació del dataset

Per instal·lar les llibreries necessàries, executarem la següent cel·la. 

Anem a utilitzar `pytorch` (una llibreria de deep learning), `pipeline` (una llibreria de tractament de dades), `scikit-learn` (una llibreria de machine learning) i `transformers` (una llibreria de models de llenguatge).

In [1]:
# Instalem les llibreries necessàries en les versions correctes

!pip install torch datasets scikit-learn transformers


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasets import load_dataset

# Carreguem el dataset. Es descarregarà automàticament i es guardarà en local. 
# Aquest dataset conté notícies de diferents categories. En aquest cas
# utilitzarem les categories World, Sports, Business i Sci/Tech.

dataset = load_dataset('ag_news')

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [3]:
print(dataset['train'][0])

print(dataset['train'].features)

classes = dataset['train'].features["label"].names
classes

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}


['World', 'Sports', 'Business', 'Sci/Tech']

Automáticament, la funció `load` ha dividit el dataset en dos conjunts: un de train i un de test. Per accedir a aquests conjunts, utilitzarem els atributs `train` i `test` de l'objecte `dataset`. Aquests atributs són objectes `tf.data.Dataset` que contenen els exemples i les etiquetes del conjunt d'entrenament i de test. Per accedir als exemples i les etiquetes, utilitzarem els atributs `data` i `label` de l'objecte `tf.data.Dataset`.

In [4]:
# Separem el dataset en conjunt d'entrenament i de test
ds_train = dataset['train']
ds_test = dataset['test']
# Vejam quants exemples hi ha en cada conjunt
print('Nombre d\'exemples de train:', len(ds_train))
print('Nombre d\'exemples de test:', len(ds_test))

Nombre d'exemples de train: 120000
Nombre d'exemples de test: 7600


Imprimim els primers 5 exemples del conjunt d'entrenament. Com podem veure, cada exemple és una notícia i la seva etiqueta.

In [5]:
# Imprimim els primers 5 exemples del conjunt d'entrenament
for w in ds_train.take(5):
    print(f"{w['label']} ({classes[w['label']]}) -> {w['text']}")

2 (Business) -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2 (Business) -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
2 (Business) -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
2 (Business) -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
2 (Business) -> Oil prices soar to

## Tokenització

La representació del text en un model de llenguatge requereix que el text sigui convertit en números. Si volem una representació a nivell de paraula, necessitem fer dues coses:

* Utilitzar un **tokenitzador** per dividir el text en **tokens**.
* Construir un **vocabulari** amb aquests tokens.

In [6]:
# Utilitzem el tokenitzador de BERT (un dels primers models de llenguatge basats en transformers) per tokenitzar les frases

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

print(tokenizer.tokenize("-- Hello, how are you doing today?"))

# Podem veure el vocabulari del tokenitzador
vocab = tokenizer.get_vocab()
print(len(vocab))

/home/carles/Documentos/notebooks/.venv/lib64/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


['-', '-', 'hello', ',', 'how', 'are', 'you', 'doing', 'today', '?']
30522


Utilitzant el tokenitzador, podem també convertir la nostra cadena tokenitzada en un conjunt de números:

In [7]:
tokenitzada = tokenizer.tokenize("-- Hello, how are you doing today?")

def encode(text):
    tk = tokenizer.tokenize(text)
    return tokenizer.convert_tokens_to_ids(tk)

print(encode("-- Hello, how are you doing today?"))

[1011, 1011, 7592, 1010, 2129, 2024, 2017, 2725, 2651, 1029]


## Representació del text

Per poder entrenar un model de xarxes neuronals, necessitem representar el text com a vectors de nombres. En aquesta pràctica, utilitzarem la representació Bag-of-Words (BoW) que consisteix en representar cada paraula com un nombre. Aquesta representació és molt senzilla i no té en compte l'ordre de les paraules ni la seva semàntica. Però és una representació que funciona prou bé en molts casos.

## Representació Bag-of-Words

Encara que el significat de les paraules no és fàcil de deduir sense poder accedir al context, en alguns casos, la representació Bag-of-Words pot ser útil. Per exemple, en el text d'una notícia, la paraula `covid` pot ser un bon indicador que la notícia parla sobre la pandèmia de la COVID-19 i la paraula `snow` pot ser un bon indicador que la notícia parla sobre el temps atmosfèric.

De les tècniques clàssiques de vectorització de text, la més senzilla és la representació Bag-of-Words (BoW). En aquesta representació, cada paraula es representa com un nombre. Per convertir un text en una representació BoW, primer creem un vector amb tants zeros com paraules hi ha en el vocabulari. Després, per cada paraula del text, incrementem en 1 el valor de la posició corresponent al vector. Per exemple, si el text és `this sentence is a test sentence`, el vector resultant seria `[1, 2, 1, 1, 0, 0, 0, 0, 0, 0, ...]`.

Si recordem la representació one-hot, veurem que la representació BoW és molt similar. La diferència és que la representació one-hot serà una sèrie de vectors amb un sol 1 i la resta de valors a 0. En canvi, la representació BoW serà un vector amb tants 1 com vegades apareixi cada paraula. Podem considerar que la representació BoW seria la suma de vectors one-hot.

Per exemple, si el text és `this sentence is a test sentence`, el vector one-hot de la primera paraula seria `[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]` i el vector one-hot de la segona paraula seria `[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...]`. La representació BoW seria la suma d'aquests dos vectors: `[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...]`.

Per generar una representació BoW, utilitzarem aquesta tècnica per convertir cada paraula en un vector one-hot i després sumarem tots els vectors. Per fer-ho, utilitzarem la funció `to_bow` que crearem a continuació. Aquesta funció rep un text i retorna un vector amb la representació BoW del text.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)

vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

Per a calcular el vector BoW de d'una noticia del nostre dataset AG_NEWS, podem utilitzar la següent funció:

In [9]:
import torch

len_vocab = len(vocab)

def to_bow(text, tamany_vocabulari=len_vocab):
    res = torch.zeros(tamany_vocabulari, dtype=torch.float32)

    for i in encode(text):
        if i<tamany_vocabulari:
            res[i] += 1
    return res

print(ds_train[0])
print(to_bow(ds_train[0]["text"]))

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
tensor([0., 0., 0.,  ..., 0., 0., 0.])


### Entrenament del model de classificació BoW

El nostre primer model serà un classificador de notícies utilitzant la representació BoW. Per fer-ho, crearem un model de xarxes neuronals amb una capa d'entrada amb tants neurones com paraules hi hagi al nostre vocabulari i una capa de sortida amb tants neurones com categories hi hagi al nostre dataset.

#### Representació BoW

En primer lloc necessitem convertir el text en la representació BoW utilitzant la funció `to_bow` que hem creat abans. Aquesta funció rep un text i retorna un vector amb la representació BoW del text.

En pytorch s'utilitzen els `DataLoaders`, per carregar les dades en lots i convertir-les en tensors de PyTorch. Aprofitarem aquesta funcionalitat per convertir les dades en BoW en tensors de PyTorch, utilitzant el paràmetre `collate_fn` del `DataLoader` i proporcionant una funció que converteixi les dades textuals en tensors de BoW.

In [15]:
from torch.utils.data import DataLoader


# this collate function gets list of batch_size tuples, and needs to
# return a pair of label-feature tensors for the whole minibatch
def bowify(batch):
    """
    Aquesta funció rep una llista de noticies i retorna un tensor amb les etiquetes
    (vector de floats) i un altre amb les notícies codificades com a BoW (matriu de floats on cada fila
    és un vector de BoW).
    """

    # Els labels són 0, 1, 2 o 3.
    # Utilitzem LongTensor perquè són enters.

    etiquetes = torch.LongTensor([noticia["label"] for noticia in batch])

    # Les notícies són tensors de BoW
    noticies = torch.stack([to_bow(noticia["text"]) for noticia in batch])

    return (etiquetes, noticies)

train_loader = DataLoader(ds_train, batch_size=16, collate_fn=bowify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=bowify)


### Model de classificació

Now let's define a simple classifier neural network that contains one linear layer. The size of the input vector equals to `vocab_size`, and output size corresponds to the number of classes (4). Because we are solving classification task, the final activation function is `LogSoftmax()`.

Ara crearem el model de classificació utilitzant PyTorch. Definirem un model senzill amb una capa lineal. La mida del vector d'entrada serà `vocab_size` i la mida de la sortida serà el nombre de classes (4). Com que estem resolent una tasca de classificació, la funció d'activació final serà `LogSoftmax()`.

In [13]:
net = torch.nn.Sequential(
    torch.nn.Linear(len(vocab), 4),
    torch.nn.LogSoftmax(dim=1)
)

# També podem definir la xarxa neuronal com a classe

class BoWClassifier(torch.nn.Module):
    def __init__(self, tamany_vocabulari, num_classes):
        # En el constructor definim les capes de la xarxa
        # crida al constructor de la classe pare
        super().__init__()

        # Capa lineal que passa de tamany_vocabulari a num_classes
        self.fc = torch.nn.Linear(tamany_vocabulari, num_classes)

        # Funció de softmax per obtenir probabilitats
        self.logsoftmax = torch.nn.LogSoftmax(dim=1)

        # Com a funció de pèrdua utilitzarem la NNLLLoss (Negative Log Likelihood Loss)
        # S'utilitza per classificació multiclasse i determina la pèrdua entre les prediccions
        # i les etiquetes

        self.loss = torch.nn.NLLLoss()

    def forward(self, ids):
        # x és el vector de BoW de la notícia
        # labels és l'etiqueta de la notícia

        # Passem x per la capa lineal
        ids = self.fc(ids)
        
        # Passem x per la funció de softmax
        ids = self.logsoftmax(ids)

        return ids
    
net = BoWClassifier(len(vocab), 4)

### Entrenament del model

Ara definirem el bucle d'entrenament estàndard de PyTorch. Com que el nostre dataset és bastant gran, per a la nostra finalitat docent entrenarem només per una època, i a vegades fins i tot per menys d'una època (especificant el paràmetre `epoch_size` ens permet limitar l'entrenament). També informarem de l'exactitud d'entrenament acumulada durant l'entrenament; la freqüència de notificació es especifica utilitzant el paràmetre `report_freq`.

Per entrenar el model, utilitzarem l'optimitzador `Adam` (ja que és un dels optimitzadors més utilitzats) i la funció de cost `CrossEntropyLoss` (ja que tenim un problema de classificació amb més de dues classes). 

In [14]:
def train_epoch(
    net,
    dataloader,
    lr=0.01,
    optimizer=None,
    loss_fn=torch.nn.NLLLoss(),
    epoch_size=None,
    report_freq=200,
):
    # Si no s'especifica un optimitzador, utilitzem Adam
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)

    # Posem la xarxa en mode training. Això activa el comportament de les capes Dropout, per exemple.
    net.train()

    # Inicialitzem les variables que ens serviran per calcular la precisió
    total_loss, acc, count, i = 0, 0, 0, 0

    # Iterem sobre el dataloader
    for labels, features in dataloader:

        # Posem els gradients a zero
        optimizer.zero_grad()

        # Calculem la sortida de la xarxa
        out = net(features)

        # Calculem la pèrdua. Aquesta funció ja aplica la softmax a la sortida.
        loss = loss_fn(out, labels)  # cross_entropy(out,labels)

        # Propaguem la pèrdua enrere. Això farà que es calculin els gradients.
        loss.backward()

        # Actualitzem els pesos de la xarxa. Això fa un pas d'optimització.
        optimizer.step()

        # Actualitzem les variables per calcular la precisió.
        total_loss += loss

        # Calculem la precisió. Per fer-ho, hem de convertir la sortida de la xarxa en etiquetes.
        # La classe amb la probabilitat més alta és la que predim com a etiqueta.
        _, predicted = torch.max(out, 1)
        acc += (predicted == labels).sum()

        # Actualitzem el comptador de mostres
        count += len(labels)

        # Mostrem la precisió cada report_freq mostres
        i += 1
        if i % report_freq == 0:
            print(f"{count}: acc={acc.item()/count}")

        # Si s'ha especificat epoch_size i ja hem processat aquest nombre de mostres, sortim del bucle.
        if epoch_size and count > epoch_size:
            break
    return total_loss.item() / count, acc.item() / count


# si volem entrenar la xarxa durant més èpoques, podem fer-ho amb un bucle
def train(
    net,
    train_loader,
    test_loader,
    lr=0.01,
    epochs=10,
    report_freq=200,
):
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    for epoch in range(epochs):
        print(f"Epoch {epoch}")
        train_loss, train_acc = train_epoch(
            net, train_loader, optimizer=optimizer, report_freq=report_freq
        )
        print(f"Train loss: {train_loss}, Train acc: {train_acc}")
    
train(net, train_loader, test_loader, epochs=5)


Epoch 0
3200: acc=0.7396875
6400: acc=0.8034375
9600: acc=0.8259375
12800: acc=0.841328125
16000: acc=0.8498125
19200: acc=0.8546354166666666
22400: acc=0.8602232142857142
25600: acc=0.8644140625
28800: acc=0.8658680555555556
32000: acc=0.86778125
35200: acc=0.8705113636363636
38400: acc=0.872109375
41600: acc=0.8735576923076923
44800: acc=0.8751116071428572
48000: acc=0.8755833333333334
51200: acc=0.8759765625
54400: acc=0.8772242647058823
57600: acc=0.8780381944444444
60800: acc=0.8776809210526316
64000: acc=0.8778125
67200: acc=0.8787946428571428
70400: acc=0.8792897727272727
73600: acc=0.88
76800: acc=0.8814713541666667
80000: acc=0.8832375
83200: acc=0.8846514423076923
86400: acc=0.8849537037037037
89600: acc=0.8851674107142857
92800: acc=0.8859051724137931
96000: acc=0.88640625
99200: acc=0.886633064516129
102400: acc=0.886943359375
105600: acc=0.887717803030303
108800: acc=0.8881341911764706
112000: acc=0.8883660714285714
115200: acc=0.888984375
118400: acc=0.8894763513513514
Tr

KeyboardInterrupt: 

El model ha aconseguit una accuracy de més de `0.9` en el conjunt d'entrenament; un nombre prou acceptable tenint en compte que hem simplificat el problema per reduïr el temps d'execució del tutorial. En un cas real, utilitzaríem totes les notícies del conjunt d'entrenament i el model seria més precís.

## Representació Word2Vec

La representació Word2Vec és una representació molt utilitzada en el processament de llenguatge natural. Aquesta representació té en compte el context de les paraules i permet fer operacions amb les paraules. Per exemple, si restem el vector de la paraula `king` i sumem el vector de la paraula `woman`, obtindrem un vector que serà molt similar al vector de la paraula `queen`.

Per generar la representació Word2Vec, utilitzarem la llibreria `gensim`. Aquesta llibreria conté molts models de representació de paraules. En aquest cas, utilitzarem el model `word2vec-google-news-300` que conté la representació Word2Vec de 3 milions de paraules i frases. 

> La primera vegada que s'executi aquesta cel·la, la funció `load` descarregarà el model d'1.5GB. Això pot trigar uns minuts. Un cop descarregat, el model es guardarà a la carpeta `/home/USUARI/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz` i no caldrà descarregar-lo de nou.
> Aquesta funció retorna un objecte `KeyedVectors` que conté la representació Word2Vec.

In [13]:
import gensim.downloader as api

w2v = api.load('word2vec-google-news-300')

Ara ja podem accedir a la representació Word2Vec de cada paraula. Per exemple, per accedir a la representació de la paraula `king`, utilitzarem la funció `get_vector` de l'objecte `KeyedVectors`.

In [14]:
w2v.get_vector('king')

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

També podem accedir a les paraules més similars a una paraula. Per exemple, per accedir a les paraules més similars a la paraula `king`, utilitzarem la funció `most_similar` de l'objecte `KeyedVectors`.

In [ ]:
for w, p in w2v.most_similar('king'):
    print(f"{w} -> {p}")

kings -> 0.7138045430183411
queen -> 0.6510956883430481
monarch -> 0.6413194537162781
crown_prince -> 0.6204220056533813
prince -> 0.6159993410110474
sultan -> 0.5864824056625366
ruler -> 0.5797567367553711
princes -> 0.5646552443504333
Prince_Paras -> 0.5432944297790527
throne -> 0.5422105193138123


El més interessant de la representació Word2Vec és que els vectors tenen una estructura matemàtica que permet fer operacions amb les paraules. Per exemple, si restem el vector de la paraula `king` i sumem el vector de la paraula `woman`, obtindrem un vector que serà molt similar al vector de la paraula `queen`.

$$ KING - MAN + WOMAN = QUEEN $$

Per fer aquesta operació, utilitzarem la funció `most_similar` de l'objecte `KeyedVectors` i li passarem els vectors de les paraules `king`, `woman` i `man`. Aquesta funció retornarà una llista amb les paraules més similars al vector resultant. Com podem veure, la paraula més similar és `queen`.

In [ ]:
w2v.most_similar(positive=['king', 'woman'], negative=['man'])[0]

('queen', 0.7118193507194519)

### Classificador Word2Vec

Ara crearem un classificador de notícies utilitzant la representació Word2Vec. En primer lloc haurem d'obtenir la representació Word2Vec de cada paraula per convertir el text en vectors. Després, sumarem tots els vectors per obtenir un vector per cada notícia. Aquest vector serà la representació de la notícia.

Per convertir un text en un vector, utilitzarem la funció `to_w2v` que crearem a continuació. Aquesta funció rep un text i retorna un vector amb la representació Word2Vec del text.

In [ ]:
def to_w2v(text):
    res = torch.zeros(300, dtype=torch.float32)
    for word in text:
        if word in w2v:
            res += torch.tensor(w2v.get_vector(word))
    return res

print(to_w2v(ds_train[0]["text"]))

tensor([-17.0809,  11.0404,  -0.9337,  12.4042,  -6.2286,   3.0224, -10.0442,
         -8.5156,  -5.9407,   1.1501,  -3.8471,  -8.0006, -18.2444,   4.3982,
        -14.2061,  11.0110,  11.2352,  14.8521,  -2.5686,   2.8961, -22.3914,
         -3.2182,   9.7872,   0.3238,  -8.6214,   4.2367, -21.9348,   5.7704,
         -0.6942,  -1.7075,  -2.4800,   2.1805,  -7.0602, -12.3824, -11.6949,
          8.2563, -18.9995,  11.3932,  -7.3198,   7.3370,  -6.1129,  -3.6244,
          5.8519,   8.3060,   3.9137,  -1.8091,  -3.2730, -15.8203,  -9.6418,
          8.9092, -16.8270,  24.5614,  -2.5387,  21.7112,   6.0571,  14.3324,
        -17.4978, -12.2693,   1.1129, -15.9192, -12.1886,  -9.5650, -19.0873,
         -7.7948,  -4.9111, -18.4653, -10.2332,  11.3437,  -6.0452,   5.4705,
          3.7500,  -9.5068,   4.4747,  -0.2912,  -3.9221,   0.3543,  13.0927,
          2.3088,   3.5300, -11.2126, -14.8031,  -2.9008,  -3.4219,  -0.3365,
         13.8353,   7.0914,  -5.2219,  22.0132,   4.2657,   5.84

Igual que hem fet amb la representació BoW, utilitzarem els `DataLoaders` de PyTorch per convertir les dades en vectors Word2Vec en tensors de PyTorch. Aprofitarem el paràmetre `collate_fn` del `DataLoader` per proporcionar una funció que converteixi les dades textuals en tensors Word2Vec.

In [18]:
def w2vify(batch):
    etiquetes = torch.LongTensor([noticia["label"] for noticia in batch])
    noticies = torch.stack([to_w2v(tokenizer.tokenize(noticia["text"])) for noticia in batch])
    return etiquetes, noticies

train_loader = DataLoader(ds_train, batch_size=16, collate_fn=w2vify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=w2vify)

### Model de classificació

Ara crearem el model de classificació utilitzant PyTorch. Definirem un model senzill amb una capa lineal. La mida del vector d'entrada serà `300` (la mida de la representació Word2Vec) i la mida de la sortida serà el nombre de classes (4). Com que estem resolent una tasca de classificació, la funció d'activació final serà `LogSoftmax()`.

In [19]:
net = torch.nn.Sequential(
    torch.nn.Linear(300, 4),
    torch.nn.LogSoftmax(dim=1)
)

Finalment, entrenarem el model utilitzant el mateix procediment que hem fet amb la representació BoW.

In [ ]:
train_epoch(net, train_loader, epoch_size=15000)

3200: acc=0.7390625
6400: acc=0.7721875
9600: acc=0.7821875
12800: acc=0.793203125


(0.08547515706466968, 0.7995735607675906)

El resultat no es molt bo. Això és perquè el model Word2Vec que hem utilitzat no té les paraules que apareixen en el dataset. Per exemple, si busquem la paraula `covid`, veurem que no apareix en el model.

Per solucionar aquest problema hauriem d'utilitzar un model Word2Vec entrenat amb les paraules del dataset. Però això és molt lent i no ho farem en aquest tutorial.